# Dependenies
Let's download and import all the needed libraries.
- **PyOWM** is needed to retrieve the weather forecast,
- **DateTime** is needed to make arithmetics on dates,
- **RasaNLU** is choosed as a framework for this task.

In [0]:
%%capture
!pip install pyowm
!pip install rasa_nlu[tensorflow]

In [0]:
import pyowm
import datetime
import rasa_nlu as nlu

# Creation of intention dataset

In [0]:
temp_intent_templates = [
    'what is the weather like {}',
    'what is the temperature {}',
    'how may degrees {}',
    'is it cold {}',
    'is it cold outside {}',
    'is it warm {}',
    'is it warm outside {}',
]

wind_intent_templates = [
    'how windy {}',
    'what is the wind {}',
    'will it blow {}',
]

rain_intent_templates = [
    'is it going to rain {}',
    'will it rain {}',
    'what is the precipitation {}',
    'should i expect rain {}'
]

day_phrases = {
    'today': 0,
    'now': 0,
    'tomorrow': 1,
    'in day': 1,
    'in one day': 1,
    'in two days': 2,
    'in three days': 3,
    'in 1 day': 1,
    'in 2 days': 2,
    'in 3 days': 3,
}

intents = '\n## intent:temp\n'
for template in temp_intent_templates:
    for phrase in day_phrases:
        entity = '[' + phrase + '](day:' + str(day_phrases[phrase]) + ')'
        intents += '- ' + (template.format(entity) + '\n')

intents += '\n## intent:wind\n'
for template in wind_intent_templates:
    for phrase in day_phrases:
        entity = '[' + phrase + '](day:' + str(day_phrases[phrase]) + ')'
        intents += '- ' + (template.format(entity) + '\n')

intents += '\n## intent:rain\n'
for template in rain_intent_templates:
    for phrase in day_phrases:
        entity = '[' + phrase + '](day:' + str(day_phrases[phrase]) + ')'
        intents += '- ' + (template.format(entity) + '\n')

with open('intents.md', 'w') as file: file.write(intents)
with open('config.yml', 'w') as file: file.write('language: "en"\npipeline: "supervised_embeddings"')

# Building chatbot

In [0]:
MY_LOCATION = 'Novosibirsk, RU'
OWM_API_KEY = 'b2a209b25b7a0486ee4c845b7e20d397'
NLU_ANSWERS = {
    'temp': 'The temperature will be {0} °C flactuating between {1} °C and {2} °C.',
    'wind': 'The wind will be {} m/s.',
    'rain': 'It will be {} outside.',
    'reject': 'Sorry, I do not understand your question.'
}

def create_intent_classifier(data_path, config_path, model_path):
    intents = nlu.training_data.load_data(data_path)
    trainer = nlu.model.Trainer(nlu.config.load(config_path))
    trainer.train(intents)
    model_dir = trainer.persist(model_path, project_name='current', fixed_model_name='nlu') 
    return nlu.model.Interpreter.load(model_dir)

class WeatherBot:
    def __init__(self):
        self.classifier = create_intent_classifier('intents.md', 'config.yml', 'model')
        self.answers = NLU_ANSWERS
        self.owm = pyowm.OWM(OWM_API_KEY)
        
    def __call__(self, question):
        markup = self.classifier.parse(question)
        if markup['intent']['confidence'] < 0.8:
            return self.answers['reject']
          
        if len(markup['entities']) != 0:
            days = int(markup['entities'][0]['value'])
        else:
            days = 0
            
        forecaster = self.owm.three_hours_forecast(MY_LOCATION)
        day = forecaster.when_starts('date') + datetime.timedelta(days=days)
        weather = forecaster.get_weather_at(day)

        intent = markup['intent']['name']
        if intent.startswith('temp'):
            temperature = weather.get_temperature(unit='celsius')
            return self.answers['temp'].format(temperature['temp'], temperature['temp_min'], temperature['temp_max'])
        elif intent.startswith('wind'):
            return self.answers['wind'].format(weather.get_wind(unit='meters_sec')['speed'])
        else:
            return self.answers['rain'].format(weather.get_status().lower())

# Chatting with WeatherBot

In [0]:
%%capture
bot = WeatherBot()

In [7]:
questions = [
    'Will it rain tomorrow?',
    'How is the weather like today?',  
    'How strong wind will be in three days?', 
    'Who killed JFK?'
]

for question in questions:
    print('Q: {}\nA: {}\n'.format(question, bot(question)))

Q: Will it rain tomorrow?
A: It will be clouds outside.

Q: How is the weather like today?
A: The temperature will be 20.65 °C flactuating between 19.63 °C and 20.65 °C.

Q: How strong wind will be in three days?
A: The wind will be 3.43 m/s.

Q: Who killed JFK?
A: Sorry, I do not understand your question.

